# Linear Regression
### 本次目標：由前 9 個小時的 18 個 features (包含 PM2.5)預測的 10 個小時的 PM2.5。

## Load 'train.csv'
### train.csv 的資料為 12 個月中，每個月取 20 天，每天 24 小時的資料(每小時資料有 18 個 features)。

In [1]:
import sys 
import pandas as pd
import numpy as np
data = pd.read_csv('train.csv', encoding = 'big5')
print(data.shape)
print(data.head())

(4320, 27)
         日期  測站        測項     0     1     2     3     4     5     6  ...  \
0  2014/1/1  豐原  AMB_TEMP    14    14    14    13    12    12    12  ...   
1  2014/1/1  豐原       CH4   1.8   1.8   1.8   1.8   1.8   1.8   1.8  ...   
2  2014/1/1  豐原        CO  0.51  0.41  0.39  0.37  0.35   0.3  0.37  ...   
3  2014/1/1  豐原      NMHC   0.2  0.15  0.13  0.12  0.11  0.06   0.1  ...   
4  2014/1/1  豐原        NO   0.9   0.6   0.5   1.7   1.8   1.5   1.9  ...   

     14    15    16    17    18    19    20    21    22    23  
0    22    22    21    19    17    16    15    15    15    15  
1   1.8   1.8   1.8   1.8   1.8   1.8   1.8   1.8   1.8   1.8  
2  0.37  0.37  0.47  0.69  0.56  0.45  0.38  0.35  0.36  0.32  
3   0.1  0.13  0.14  0.23  0.18  0.12   0.1  0.09   0.1  0.08  
4   2.5   2.2   2.5   2.3   2.1   1.9   1.5   1.6   1.8   1.5  

[5 rows x 27 columns]


In [2]:
from IPython.display import Image
from IPython.core.display import HTML 

## Preprocessing
### 取需要的數值部分，將 'RAINFALL' 欄位全部補 0。如要重覆這段程式碼的執行，請從頭開始執行(把上面的都重新跑一次)，以避免跑出不是自己要的結果（若自己寫程式不會遇到，但 colab 重複跑這段會一直往下取資料。意即第一次取原本資料的第三欄之後的資料，第二次取第一次取的資料掉三欄之後的資料，...）。 

In [3]:
data  = data.iloc[:, 3:]
data[data == 'NR'] = 0
raw_data = data.to_numpy()
print(raw_data.shape)

(4320, 24)


# **Extract Features (1)**
![Image of Yaktocat](https://drive.google.com/uc?id=1LyaqD4ojX07oe5oDzPO99l9ts5NRyArH)
![Image of Yaktocat](https://drive.google.com/uc?id=1ZroBarcnlsr85gibeqEF-MtY13xJTG47)
將原始 4320 * 24 的資料依照每個月分重組成 12 個 18 (features) * 480 (hours) 的資料。 

In [4]:
month_data = {}
for month in range(12):
    sample = np.empty([18, 480])
    for day in range(20):
        sample[:, day * 24 : (day + 1) * 24] = raw_data[18 * (20 * month + day) : 18 * (20 * month + day + 1), :]
        # 儲存至 (每個月)[x , 480] 
        # raw_data => 因一天的資料有18個feature 故要使用 18 * (20 * month + day) ， 且取列
    month_data[month] = sample

# **Extract Features (2)**
![Image of Yaktocat](https://drive.google.com/uc?id=1wKoPuaRHoX682LMiBgIoOP4PDyNKsJLK)
![Image of Yaktocat](https://drive.google.com/uc?id=1FRWWiXQ-Qh0i9tyx0LiugHYF_xDdkhLN)

每個月會有 480hrs，每 9 小時形成一個 data，每個月會有 471 個 data(一個一個平移)，故總資料數為 471 * 12 筆，而每筆 data 有 9 * 18 的 features (一小時 18 個 features * 9 小時)。

對應的 target 則有 471 * 12 個(第 10 個小時的 PM2.5)



In [5]:
print(type(month_data))
x = np.empty([ 471 * 12 , 18 * 9] , dtype = float)
y = np.empty([471 * 12 , 1] , dtype = float)
for month in range(12):
    for day in range(20):
        for hour in range(24):
            if day == 19 and hour > 14 :
                continue
            x[month * 471 + day * 24 + hour, :] = month_data[month][:,day * 24 + hour : day * 24 + hour + 9].reshape(1, -1) #vector dim:18*9 (9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9)
            y[month * 471 + day * 24 + hour, 0] = month_data[month][9, day * 24 + hour + 9] #value
print(x.shape)
print(y.shape)

<class 'dict'>
(5652, 162)
(5652, 1)


## Normalize (1)

In [6]:
mean_x = np. mean(x , axis = 0) # 18*9
print(mean_x.shape)
std_x = np.std(x , axis = 0) #18 * 9
for i in range(len(x)): # 471*12
    for j in range(len(x[0])) :#18*9
        if std_x[j] != 0 :
            x[i][j] = (x[i][j] - mean_x[j]) / std_x[j]
print(x)

(162,)
[[-1.35825331 -1.35883937 -1.359222   ...  0.26650729  0.2656797
  -1.14082131]
 [-1.35825331 -1.35883937 -1.51819928 ...  0.26650729 -1.13963133
  -1.32832904]
 [-1.35825331 -1.51789368 -1.67717656 ... -1.13923451 -1.32700613
  -0.85955971]
 ...
 [-0.88092053 -0.72262212 -0.56433559 ... -0.57693779 -0.29644471
  -0.39079039]
 [-0.7218096  -0.56356781 -0.72331287 ... -0.29578943 -0.39013211
  -0.1095288 ]
 [-0.56269867 -0.72262212 -0.88229015 ... -0.38950555 -0.10906991
   0.07797893]]


In [7]:
import math
# math.floor()會傳回輸入值的最大整數
x_train_set = x[: math.floor(len(x) * 0.8) ,:]
y_train_set = y[: math.floor(len(y) * 0.8) ,:]
x_validation = x[math.floor(len(x) * 0.8) : ,:]
y_validation = y[math.floor(len(y) * 0.8) : ,:]
print(len(x_train_set))
print(len(y_train_set))
print(len(x_validation))
print(len(y_validation))

4521
4521
1131
1131


# **Training**
![Image of Yaktocat](https://drive.google.com/uc?id=1xIXvqZ4EGgmxrp7c9r0LOVbcvd4d9H4N)
![Image of Yaktocat](https://drive.google.com/uc?id=1S42g06ON5oJlV2f9RukxawjbE4NpsaB6)
![Image of Yaktocat](https://drive.google.com/uc?id=1BbXu-oPB9EZBHDQ12YCkYqtyAIil3bGj)

(和上圖不同處: 下面的 code 採用 Root Mean Square Error)

因為常數項的存在，所以 dimension (dim) 需要多加一欄；eps 項是避免 adagrad 的分母為 0 而加的極小數值。

每一個 dimension (dim) 會對應到各自的 gradient, weight (w)，透過一次次的 iteration (iter_time) 學習。

In [8]:
dim = 18 * 9 + 1
w = np.zeros([dim, 1])
x = np.concatenate((np.ones([12 * 471, 1]), x), axis = 1).astype(float)
learning_rate = 100
iter_time = 1000
adagrad = np.zeros([dim, 1])
eps = 0.0000000001
for t in range(iter_time):
    loss = np.sqrt(np.sum(np.power(np.dot(x, w) - y, 2))/471/12) #Root Mean Square Error
    # np.power(x,y) => x^y
    if(t%100==0):
        print(str(t) + ":" + str(loss))
    gradient = 2 * np.dot(x.transpose(), np.dot(x, w) - y) #dim*1
    adagrad += gradient ** 2
    w = w - learning_rate * gradient / np.sqrt(adagrad + eps)
np.save('weight.npy', w)
w

0:27.071214829194115
100:33.78905859777454
200:19.9137512981971
300:13.53106819368969
400:10.645466158446167
500:9.27735345547506
600:8.518042045956495
700:8.014061987588416
800:7.636756824775687
900:7.336563740371119


array([[ 2.13740269e+01],
       [ 3.58888909e+00],
       [ 4.56386323e+00],
       [ 2.16307023e+00],
       [-6.58545223e+00],
       [-3.38885580e+01],
       [ 3.22235518e+01],
       [ 3.49340354e+00],
       [-4.60308671e+00],
       [-1.02374754e+00],
       [-3.96791501e-01],
       [-1.06908800e-01],
       [ 2.22488184e-01],
       [ 8.99634117e-02],
       [ 1.31243105e-01],
       [ 2.15894989e-02],
       [-1.52867263e-01],
       [ 4.54087776e-02],
       [ 5.20999235e-01],
       [ 1.60824213e-01],
       [-3.17709451e-02],
       [ 1.28529025e-02],
       [-1.76839437e-01],
       [ 1.71241371e-01],
       [-1.31190032e-01],
       [-3.51614451e-02],
       [ 1.00826192e-01],
       [ 3.45018257e-01],
       [ 4.00130315e-02],
       [ 2.54331382e-02],
       [-5.04425219e-01],
       [ 3.71483018e-01],
       [ 8.46357671e-01],
       [-8.11920428e-01],
       [-8.00217575e-02],
       [ 1.52737711e-01],
       [ 2.64915130e-01],
       [-5.19860416e-02],
       [-2.5

# **Testing**
![Image of Yaktocat](https://drive.google.com/uc?id=1165ETzZyE6HStqKvgR0gKrJwgFLK6-CW)

載入 test data，並且以相似於訓練資料預先處理和特徵萃取的方式處理，使 test data 形成 240 個維度為 18 * 9 + 1 的資料。

In [12]:
test_data = pd.read_csv('test.csv', header = None , encoding = 'big5')
test_data = test_data.iloc[:, 2:]
test_data[test_data == 'NR'] = 0
test_data = test_data.to_numpy()
test_x = np.empty([240,  18 * 9] , dtype = float)
for i in range(240):
    test_x[i, :] = test_data[18 * i : 18 * (i + 1) , :].reshape(1, -1)
for i in range(len(test_x)):
    for j in range(len(test_x[0])):
        if std_x[j] != 0:
            test_x[i][j] = ( test_x[i][j] - mean_x[j]) / std_x[j]
test_x = np.concatenate((np.ones([240 , 1]) , test_x ), axis = 1).astype(float)
# np.concatenate((np.ones([240 , 1]) , test_x ) 新增一行 1. 在 test_x前面
print(test_x.shape)
print(test_x)

(240, 163)
[[ 1.         -0.24447681 -0.24545919 ... -0.67065391 -1.04594393
   0.07797893]
 [ 1.         -1.35825331 -1.51789368 ...  0.17279117 -0.10906991
  -0.48454426]
 [ 1.          1.5057434   1.34508393 ... -1.32666675 -1.04594393
  -0.57829812]
 ...
 [ 1.          0.3919669   0.54981237 ...  0.26650729 -0.20275731
   1.20302531]
 [ 1.         -1.8355861  -1.8360023  ... -1.04551839 -1.13963133
  -1.14082131]
 [ 1.         -1.35825331 -1.35883937 ...  2.98427476  3.26367657
   1.76554849]]


# **Prediction**
說明圖同上

![Image of Yaktocat](https://drive.google.com/uc?id=1165ETzZyE6HStqKvgR0gKrJwgFLK6-CW)
有了 weight 和測試資料即可預測 target。

In [14]:
w = np.load('weight.npy')
ans_y = np.dot(test_x,w)
ans_y

array([[ 5.17496040e+00],
       [ 1.83062143e+01],
       [ 2.04912181e+01],
       [ 1.15239429e+01],
       [ 2.66160568e+01],
       [ 2.05313481e+01],
       [ 2.19065510e+01],
       [ 3.17364687e+01],
       [ 1.33916741e+01],
       [ 6.44564665e+01],
       [ 2.02645688e+01],
       [ 1.53585761e+01],
       [ 6.85894728e+01],
       [ 4.84281137e+01],
       [ 1.87023338e+01],
       [ 1.01885957e+01],
       [ 3.07403629e+01],
       [ 7.11322178e+01],
       [-4.13051739e+00],
       [ 1.82356940e+01],
       [ 3.85789223e+01],
       [ 7.13115197e+01],
       [ 7.41034816e+00],
       [ 1.87179553e+01],
       [ 1.49372503e+01],
       [ 3.67197367e+01],
       [ 1.79616970e+01],
       [ 7.57894629e+01],
       [ 1.23093102e+01],
       [ 5.62953517e+01],
       [ 2.51131609e+01],
       [ 4.61024867e+00],
       [ 2.48377055e+00],
       [ 2.47594223e+01],
       [ 3.04802805e+01],
       [ 3.84639307e+01],
       [ 4.42023106e+01],
       [ 3.00868360e+01],
       [ 4.0

In [18]:
import csv
with open('submit.csv' , mode = 'w', newline = '') as submit_file:
    csv_writer = csv.writer(submit_file)
    header = ['id' , 'value']
    csv_writer.writerow(header)
    for i in range(240):
        row = ['id_' + str(i) , ans_y[i][0]]
        csv_writer.writerow(row)
        print(row)

['id_0', 5.174960398984737]
['id_1', 18.306214253527912]
['id_2', 20.49121809418056]
['id_3', 11.523942869805367]
['id_4', 26.616056752306154]
['id_5', 20.531348081761223]
['id_6', 21.906551018797394]
['id_7', 31.736468747068823]
['id_8', 13.391674055111714]
['id_9', 64.45646650291955]
['id_10', 20.264568836159448]
['id_11', 15.358576077361235]
['id_12', 68.58947276926723]
['id_13', 48.42811374745718]
['id_14', 18.702333824193197]
['id_15', 10.188595737466706]
['id_16', 30.740362859820426]
['id_17', 71.13221776355111]
['id_18', -4.130517391262437]
['id_19', 18.23569401642868]
['id_20', 38.57892227500777]
['id_21', 71.3115197253133]
['id_22', 7.410348162634051]
['id_23', 18.717955330321402]
['id_24', 14.937250260084571]
['id_25', 36.71973669470532]
['id_26', 17.9616970056627]
['id_27', 75.78946287210542]
['id_28', 12.309310248614455]
['id_29', 56.29535173964959]
['id_30', 25.11316086566149]
['id_31', 4.610248674094045]
['id_32', 2.483770554515047]
['id_33', 24.759422261321276]
['id_34',

相關 reference 可以參考:

Adagrad :
https://youtu.be/yKKNr-QKz2Q?list=PLJV_el3uVTsPy9oCRY30oBPNLCo89yu49&t=705 

RMSprop : 
https://www.youtube.com/watch?v=5Yt-obwvMHI 

Adam
https://www.youtube.com/watch?v=JXQT_vxqwIs 


以上 print 的部分主要是為了看一下資料和結果的呈現，拿掉也無妨。另外，在自己的 linux 系統，可以將檔案寫死的的部分換成 sys.argv 的使用 (可在 terminal 自行輸入檔案和檔案位置)。

最後，可以藉由調整 learning rate、iter_time (iteration 次數)、取用 features 的多寡(取幾個小時，取哪些特徵欄位)，甚至是不同的 model 來超越 baseline。

Report 的問題模板請參照 : https://docs.google.com/document/d/1s84RXs2AEgZr54WCK9IgZrfTF-6B1td-AlKR9oqYa4g/edit